In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import os

In [2]:
df = pd.read_csv("../features/features.csv", sep=',', header=None)

In [3]:
df.isnull().sum().sum()

0

In [4]:
df = df.fillna(0)

In [5]:
no_nulls = df.apply(lambda row : 
          any([ e == ("null") for e in row ])
       , axis=1) 

In [6]:
no_nulls[597]

True

In [7]:
df.loc[[597]]

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
597,purple,1599765459,null,null,null,null,null,null,null,null,...,25669.053,441.94736,4.5789475,5.868421,0.05263158,0.05526315,236.07895,369.8,0.05263158,0.05263158


In [8]:
df = df[~no_nulls]
df = df.apply(pd.to_numeric, errors='ignore')


In [9]:
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,162,163,164,165,166,167,168,169,170,171
0,blue,1590361949,0.500000,394.38733,1.884146,0.884146,2.550000,148904.95,21593.250,26485.550,...,30334.200,1902.20000,3.960000,5.550000,0.050000,0.662500,238.48001,407.56000,0.050000,38.792503
1,blue,1604941176,0.421053,358.31810,1.705357,0.705357,1.947368,90007.52,13427.842,17912.000,...,22437.300,298.20000,1.080000,1.530000,0.055000,0.177500,167.64499,298.63500,9.492499,0.050000
2,blue,1604313975,0.526316,353.67276,1.945946,0.945946,2.368421,74615.42,12151.263,20228.000,...,28976.790,415.05264,0.952632,1.363158,0.057895,0.052632,211.42632,326.96313,11.452632,0.052632
3,purple,1595899540,0.736842,413.60680,2.380000,1.380000,3.631579,144662.05,18813.578,19041.158,...,25255.790,358.36840,1.136842,1.357895,0.052632,0.052632,201.98947,343.47370,0.052632,0.052632
4,blue,1604303561,0.526316,373.19904,2.037037,1.037037,2.736842,142241.31,20665.000,20751.105,...,26728.578,363.21054,3.173684,3.484210,0.052632,0.052632,199.08948,316.16318,0.052632,0.052632


In [22]:
#df = df.replace(to_replace='null', value=0)
#df = df.replace(to_replace='infinity', value=0)

In [10]:
#Global constants
seed = 42
validation_size = 100
feature_count = df.shape[1] - 2

#feed forward neural net
n_nodes_hl1 = 150
n_nodes_hl2 = 150
n_nodes_hl3 = 150

#cycles of feed forward + backprop on all K-folded samples
hm_epochs = 50

n_classes = 2
batch_size = 8

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

In [23]:
import pickle

X = np.asarray(df.ix[:,2:feature_count+2])
#standardize X
meanX = np.mean(X, axis = 0)
stdX = np.std(X, axis = 0)

f = open('mean.pckl', 'wb')
pickle.dump(meanX, f)
f.close()

f = open('std.pckl', 'wb')
pickle.dump(stdX, f)
f.close()

X = (X - meanX) / stdX

Y_1 = np.asarray(df.ix[:,0])
Y_1 = [int(y == "purple") for y in Y_1]
#one hot Y
Y = np.zeros(shape=(len(Y_1), n_classes))
Y[np.arange(len(Y_1)), Y_1] = 1
    
validation_features = X[:validation_size]
validation_labels = Y[:validation_size]

train_features = X[validation_size:]
train_labels = Y[validation_size:]

num_examples = train_features.shape[0]

170

In [24]:
def neural_network_model(data):
    
    hidden_1_layer = {
        'weights': tf.Variable(tf.truncated_normal([feature_count, n_nodes_hl1], stddev=0.1, seed=seed)),
        'biases': tf.Variable(tf.constant(1.0, shape=[n_nodes_hl1]))
    }
    
    hidden_2_layer = {
        'weights': tf.Variable(tf.truncated_normal([n_nodes_hl1, n_nodes_hl2], stddev=0.1, seed=seed)),
        'biases': tf.Variable(tf.constant(1.0, shape=[n_nodes_hl2]))
    }
    
    hidden_3_layer = {
        'weights': tf.Variable(tf.truncated_normal([n_nodes_hl2, n_nodes_hl3], stddev=0.1, seed=seed)),
        'biases': tf.Variable(tf.constant(1.0, shape=[n_nodes_hl3]))
    }
    
    output_layer = {
        'weights': tf.Variable(tf.truncated_normal([n_nodes_hl3, n_classes], stddev=0.1, seed=seed)),
        'biases': tf.Variable(tf.constant(1.0, shape=[n_classes]))
    }
    
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.sigmoid(l3)
    
    output = tf.matmul(l3, output_layer['weights']) +  output_layer['biases']
    
    regularizers = (tf.nn.l2_loss(hidden_1_layer['weights']) + tf.nn.l2_loss(hidden_1_layer['biases']) +
                        tf.nn.l2_loss(hidden_2_layer['weights']) + tf.nn.l2_loss(hidden_2_layer['biases']) +
                            tf.nn.l2_loss(hidden_3_layer['weights']) + tf.nn.l2_loss(hidden_3_layer['biases']) +
                                tf.nn.l2_loss(output_layer['weights']) + tf.nn.l2_loss(output_layer['biases']))
    
    return output, regularizers

In [25]:
def train_neural_network_CV(x, lambda_):
    
    prediction, regularizers = neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
    #log_loss = tf.contrib.losses.log_loss(predictions=prediction, labels=y)

    #Eval this to get probability of [winning,losing]
    prob = tf.nn.softmax(prediction)
    
    #learning rate can be passed
    optimizer = tf.train.AdamOptimizer().minimize(cost + lambda_ * regularizers)
    
    #metrics
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    false_prediction = tf.logical_not(correct_prediction)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

    true_positives = tf.reduce_sum(tf.to_int32(tf.logical_and(correct_prediction, tf.equal(tf.argmax(tf.nn.softmax(y),1), True) )))
    false_positives = tf.reduce_sum(tf.to_int32(tf.logical_and(false_prediction, tf.equal(tf.argmax(tf.nn.softmax(y),1), True) )))
    true_negatives = tf.reduce_sum(tf.to_int32(tf.logical_and(correct_prediction, tf.equal(tf.argmax(tf.nn.softmax(y),1), False) )))
    false_negatives = tf.reduce_sum(tf.to_int32(tf.logical_and(false_prediction, tf.equal(tf.argmax(tf.nn.softmax(y),1), False) )))


    display_step = 1
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            #epoch_loss = 0
            
            #train and acc summed over all folds
            test_accs = []
            train_accs = []
            
            kf = KFold(n_splits=10, random_state=seed, shuffle=True)
            for train_index, test_index in kf.split(train_features, train_labels):
                X_train, X_test = train_features[train_index], train_features[test_index]
                y_train, y_test = train_labels[train_index], train_labels[test_index]
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: X_train, y: y_train})
                #epoch_loss += c
                
                train_accuracy = accuracy.eval(feed_dict={ x: X_train, y: y_train})  
                test_accuracy = accuracy.eval(feed_dict={ x: X_test, y: y_test})  
                
                
                train_accs.append(train_accuracy)
                test_accs.append(test_accuracy)
                
            train_acc = np.sum(train_accs) / len(train_accs)
            test_acc = np.sum(test_accs) / len(test_accs)
            
            # increase display_step after 10 iteration of same decimal
            if epoch%(display_step*10) == 0 and epoch:
                display_step *= 10
                
            if epoch%display_step == 0 or (epoch+1) == hm_epochs:
               
                #prob_test = validation_features[0].reshape((1,validation_features[0].shape[0]))
                #prob_value = prob.eval(feed_dict={ x:prob_test})  
                #print('probability test', prob_value)
                
                validation_accuracy = accuracy.eval(feed_dict={ x: validation_features, y: validation_labels})     
                print('train accuracy => %.2f, test acc =>  %.2f, validation accuracy => %.2f for epoch %d' % (train_acc, test_acc, validation_accuracy, epoch))

        saver = tf.train.Saver()
        save_path = saver.save(sess, model_path)        

In [26]:
x = tf.placeholder('float', [None, feature_count])
y = tf.placeholder('float', [None, n_classes])

model_path = "./tmp/model.ckpt"
save_dir = './tmp/'
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

L2_lambda_ = 5e-5
train_neural_network_CV(x, L2_lambda_)

train accuracy => 0.54, test acc =>  0.54, validation accuracy => 0.45 for epoch 0
train accuracy => 0.60, test acc =>  0.60, validation accuracy => 0.42 for epoch 1
train accuracy => 0.63, test acc =>  0.63, validation accuracy => 0.47 for epoch 2
train accuracy => 0.66, test acc =>  0.66, validation accuracy => 0.49 for epoch 3
train accuracy => 0.69, test acc =>  0.69, validation accuracy => 0.48 for epoch 4
train accuracy => 0.73, test acc =>  0.73, validation accuracy => 0.47 for epoch 5
train accuracy => 0.77, test acc =>  0.77, validation accuracy => 0.47 for epoch 6
train accuracy => 0.82, test acc =>  0.81, validation accuracy => 0.45 for epoch 7
train accuracy => 0.87, test acc =>  0.87, validation accuracy => 0.41 for epoch 8
train accuracy => 0.92, test acc =>  0.92, validation accuracy => 0.44 for epoch 9
train accuracy => 0.96, test acc =>  0.96, validation accuracy => 0.46 for epoch 10
train accuracy => 1.00, test acc =>  1.00, validation accuracy => 0.45 for epoch 20
tr

In [28]:
# Running a new session to predict based on model
#TODO make sure it works and test
prediction, regularizers = neural_network_model(x)
#Eval this to get probability of [winning,losing]
prob = tf.nn.softmax(prediction)

with tf.Session() as sess:
    # Initialize variables
    sess.run(tf.global_variables_initializer())
    
#     saver.restore(sess, model_path)
    new_saver = tf.train.import_meta_graph(model_path + ".meta")
    new_saver.restore(sess, tf.train.latest_checkpoint('./tmp'))
    
    #test random sample from validation test
    prob_test = validation_features[40].reshape((1,validation_features[0].shape[0]))
    prob_value = prob.eval(feed_dict={ x:prob_test})  
    print('probability test', prob_value)
    
    

probability test [[ 0.38545015  0.61454988]]


In [29]:
prob_test[0]

array([-0.32692872, -0.31204851, -0.32684987, -0.32688453, -0.32676454,
        5.21943154,  0.50826235,  0.38402118, -0.31375633, -0.32673353,
       -0.32671255, -0.32694888, -0.32693146, -0.31751632, -0.31180154,
       -0.32656407, -0.32534424, -0.32692872, -0.31120542, -0.32684049,
       -0.32687515, -0.32676819,  8.44878065,  0.94904214,  0.38893186,
       -0.30494007, -0.32671912, -0.32670471, -0.32695061, -0.3269403 ,
       -0.3168691 , -0.31103848, -0.32660903, -0.32516702, -0.32693602,
       -0.31333459, -0.32688257, -0.32691723, -0.32682657,  3.58025366,
        0.43102299,  0.67482447, -0.31463558, -0.32677403, -0.32673244,
       -0.32695061, -0.32691869, -0.31811666, -0.31209724, -0.32695061,
       -0.32602666, -0.32692689, -0.31192594, -0.32686835, -0.32690301,
       -0.32682292,  5.78753747,  0.55221774,  0.5592773 , -0.30653076,
       -0.3267972 , -0.32680522, -0.32692735, -0.32695061, -0.3169428 ,
       -0.31257845, -0.32593181, -0.32630248, -0.3269351 , -0.31